# 线性规划的标准型  
(1)求目标函数的最大值   
(2)约束条件为等式约束   
(3)约束条件右边的常数项大于或等于0  
(4)所有变量大于或等于0  
## 非标准型向标准型转化 
添加松弛变量使不等式成为等式    
(1)目标函数求最小,两边同乘-1    
(2)约束方程右边为负数, 乘-1   
(3)变量没有约束,则变量写作 $x_{1}=x_{1}^{'}-x_{1}^{''}$,$x_{1}^{'}>=0, x_{1}^{''}>=0$  

# 一、单纯形法      
遍历可行域的顶点        
## 使用条件     
成为规范型      
（1）标准型     
（2）约束方程组的系数矩阵中至少有一个单位子矩阵         
        构成单位子矩阵的系数对应的变量叫做基变量，剩下的叫非基变量      
（3）目标函数不含非基变量       
## 计算过程     
（1）确定初始可行解，令基变量=0         
（2）判断当前点X是否为最优解    
        通过判断目标中的变量的系数是否为负数                                                                                                  
（3）基变量出基与非基变量入基     
        不同行进行加减  
（4）计算新的解         
        依次循环...     


In [15]:
import pandas as pd
from pandas import DataFrame
import numpy as np
# 定义线性规划求解函数
def lp_solver(matrix: DataFrame):
    """
    输入线性规划的矩阵，根据单纯形法求解线性规划模型
     max cx
    s.t. ax<=b
    矩阵形式是：
             b    x1    x2   x3   x4   x5
    obj    0.0  70.0  30.0  0.0  0.0  0.0
    x3   540.0   3.0   9.0  1.0  0.0  0.0
    x4   450.0   5.0   5.0  0.0  1.0  0.0
    x5   720.0   9.0   3.0  0.0  0.0  1.0
    第1行是目标函数的系数
    第2-4行是约束方程的系数
    第1列是约束方程的常数项
    obj-b 交叉，即第1行第1列的元素是目标函数的负值
    x3,x4,x5 是松弛变量，也是初始可行解
    :param matrix:
    :return:
    """
    # 检验数是否大于0
    c = matrix.iloc[0, 1:]
    
    while c.max() > 0:
        # 这里指的是约束方程的系数
        # 选择入基变量，目标函数系数最大的变量入基
        c = matrix.iloc[0, 1:]
        print('c:\n',c)
        in_x = c.idxmax()
        print('in_x:\n',in_x)
        in_x_v = c[in_x]  # 入基变量的系数
        # 选择出基变量
        # 选择正的最小比值对应的变量出基 min( b列/入基变量列)
        b = matrix.iloc[1:, 0]
        print('b:\n', b)
        in_x_a = matrix.iloc[1:][in_x] 
        print('in_x_a:\n',in_x_a)
         # 选择入基变量对应的列
        out_x = (b / in_x_a).idxmin()  # 得到出基变量
        print('out_x',out_x)
        # 旋转操作
        matrix.loc[out_x, :] = matrix.loc[out_x, :] / matrix.loc[out_x, in_x]
        for idx in matrix.index:
            if idx != out_x:
                matrix.loc[idx, :] = matrix.loc[idx, :] - matrix.loc[out_x, :] * matrix.loc[idx, in_x]
        # 索引替换（入基出基变量名称替换）
        index = matrix.index.tolist()
        i = index.index(out_x)
        index[i] = in_x
        matrix.index = index
    # 打印结果
    print("最终的最优单纯形法是：")
    print(matrix)
    print("目标函数值是：", - matrix.iloc[0, 0])
    print("最优决策变量是：")
    x_count = (matrix.shape[1] - 1) - (matrix.shape[0] - 1)
    X = matrix.iloc[0, 1:].index.tolist()[: x_count]
    for xi in X:
        print(xi, '=', matrix.loc[xi, 'b'])


In [16]:
# 主程序代码
def main():
    # 约束方程系数矩阵，包含常数项
    matrix = pd.DataFrame(
        np.array([
            [0, 70, 30, 0, 0, 0],
            [540, 3, 9, 1, 0, 0],
            [450, 5, 5, 0, 1, 0],
            [720, 9, 3, 0, 0, 1]]),
        index=['obj', 'x3', 'x4', 'x5'],
        columns=['b', 'x1', 'x2', 'x3', 'x4', 'x5'])
    # 调用前面定义的函数求解
    lp_solver(matrix)
main()

c:
 x1    70
x2    30
x3     0
x4     0
x5     0
Name: obj, dtype: int32
in_x:
 x1
b:
 x3    540
x4    450
x5    720
Name: b, dtype: int32
in_x_a:
 x3    3
x4    5
x5    9
Name: x1, dtype: int32
out_x x5
c:
 x1    0.000000
x2    6.666667
x3    0.000000
x4    0.000000
x5   -7.777778
Name: obj, dtype: float64
in_x:
 x2
b:
 x3    300
x4     50
x1     80
Name: b, dtype: int32
in_x_a:
 x3    8.000000
x4    3.333333
x1    0.333333
Name: x2, dtype: float64
out_x x4
c:
 x1    0.000000
x2    0.000000
x3    0.000000
x4   -2.000000
x5   -6.666667
Name: obj, dtype: float64
in_x:
 x1
b:
 x3    180
x2     15
x1     75
Name: b, dtype: int32
in_x_a:
 x3    0
x2    0
x1    1
Name: x1, dtype: int32
out_x x1
最终的最优单纯形法是：
        b  x1   x2  x3   x4        x5
obj -5700   0  0.0   0 -2.0 -6.666667
x3    180   0  0.0   1 -2.4  1.000000
x2     15   0  1.0   0  0.3 -0.166667
x1     75   1  0.0   0 -0.1  0.166667
目标函数值是： 5700
最优决策变量是：
x1 = 75
x2 = 15


# 二、内点法    
在大规模计算上有比较好的表现           
在可行域内求解，因此处理的为不等式约束        
$ min Z=c^{T}x $    
$s.t. Ax<=b$        
借助拉格朗日松弛法的思路，表示成：      
$ min f(x)=c^{T}x+\sum_{i = 1}^{m}I(Ax-b)$      
m 为约束方程的个数，I（x）为指数函数         
$
f(x) = 
\begin{cases}
0, x<=0\\
\infty ,x>0\\
\end{cases}
$       
但这里的指示函数I(x)并不可导，用$I_{u}=-\frac{1}{t}\log(-Ax+b)$近似，t为超参数      


In [ ]:
import numpy as np
import time

def gradient(x1, x2, t):
    """计算目标函数在x处的一阶导数（雅克比矩阵）"""
    j1 = -70 * t + 3 / (-3 * x1 - 9 * x2 + 540) + 5 / (-5 * x1 - 5 * x2 + 450) + 9 / (-9 * x1 - 3 * x2 + 720) - 1 / x1
    j2 = -30 * t + 9 / (-3 * x1 - 9 * x2 + 540) + 5 / (-5 * x1 - 5 * x2 + 450) + 3 / (-9 * x1 - 3 * x2 + 720) - 1 / x2
    return np.asmatrix([j1, j2]).T

def hessian(x1, x2):
    """计算目标函数在x处的二阶导数（海塞矩阵）"""
    x1, x2 = float(x1), float(x2)
    h11 = 9 / (3 * x1 + x2 - 240) ** 2 + (x1 + 3 * x2 - 180) ** (-2) + (x1 + x2 - 90) ** (-2) + x1 ** (-2)
    h12 = 3 / (3 * x1 + x2 - 240) ** 2 + 3 / (x1 + 3 * x2 - 180) ** 2 + (x1 + x2 - 90) ** (-2)
    h21 = 3 / (3 * x1 + x2 - 240) ** 2 + 3 / (x1 + 3 * x2 - 180) ** 2 + (x1 + x2 - 90) ** (-2)
    h22 = (3 * x1 + x2 - 240) ** (-2) + 9 / (x1 + 3 * x2 - 180) ** 2 + (x1 + x2 - 90) ** (-2) + x2 ** (-2)
    return np.asmatrix([[h11, h12], [h21, h22]])

def invertible(H):
    """求海塞矩阵的逆矩阵"""
    H_1 = np.linalg.inv(H)
    return H_1

def main():
    x = np.asmatrix(np.array([10, 10])).T  # x 是牛顿法的初始迭代值
    t = 0.00001  # t 是指示函数的中的t
    eps = 0.01  # 迭代停止的误差
    iter_cnt = 0  # 记录迭代的次数
    while iter_cnt < 20:
        iter_cnt += 1
        J = gradient(x[0, 0], x[1, 0], t)
        H = hessian(x[0, 0], x[1, 0])
        H_1 = np.linalg.inv(H)  # 还塞矩阵的逆
        x_new = x - H_1 * J  # 牛顿法公式
        error = np.linalg.norm(x_new - x)  # 求2范数，判断迭代效果
        print('迭代次数是：%d, x1=%.2f, x2=%.2f, 误差是：%.2f' % (iter_cnt, x_new[0, 0], x_new[1, 0], error))
        x = x_new
        if error < eps:
            break
        time.sleep(1)
    # 打印结果
    print("目标函数值是：%.2f" % float(70 * x[0, 0] + 30 * x[1, 0]))
main()

# 三、列生成法      
改变单纯形法中计算效率低的问题      
用于大规模计算  
前j个为基变量   
后n-j个为非基变量   
逐步把非基变量变为基变量    

# 四、对偶问题  
$max z = cx$  $min D=b^{T}Y $      
$
\begin{cases}
Ax<=b, \\
x>=0 ,\\
\end{cases} $
$ 
\begin{cases}
A^{T}Y>=c^{T}, \\
Y>0 ,\\
\end{cases}
$       
$z=cx<=D=b^{T}Y$    
cx=D时，最优解


# 五、拉格朗日乘子法
$ min f(x)$     
$s.t.\begin{cases}
h_{i}^{x}=0, i=1,2,...,m \\
g_{j}^{x}<=0, j=1,2,...,n\\
\end{cases}
$       
转化为无约束优化问题
$L(x, \alpha, \beta)=f(x)+\sum_{i=1}^{m}\alpha_{i}h_{i}(x) + \sum_{i=1}^{n}\beta_{j}g_{j}(x)$       
可行解的KKT条件         
$ \nabla_{x}L(x, \alpha, \beta) = 0$ (1)    
$ h_{i}(x) = 0$ (2)   
$ g_{j}(x) <= 0$ (3)   
$ \beta_{j} >= 0$ (4)   
$ \beta_{j}g_{j}(x) = 0$ (5)   